In [1]:
from datetime import date, timedelta
import datetime
import pandas as pd
import numpy as np
from tqdm import tqdm, tnrange

from sklearn.metrics import mean_squared_error
import lightgbm as lgb

import mlflow
import mlflow.sklearn

from config import (
    RAW_DATA_DIR,
    FEATURE_DIR,
    LAG_DICT,
    SLIDING_DICT
)

/Users/liuyu/anaconda3/lib/python3.7/site-packages/lightgbm/__init__.py:46: UserWarning: Starting from version 2.2.1, the library file in distribution wheels for macOS is built by the Apple Clang (Xcode_8.3.3) compiler.
This means that in case of installing LightGBM from PyPI via the ``pip install lightgbm`` command, you don't need to install the gcc compiler anymore.
Instead of that, you need to install the OpenMP library, which is required for running LightGBM on the system with the Apple Clang compiler.
You can install the OpenMP library by the following command: ``brew install libomp``.
  "You can install the OpenMP library by the following command: ``brew install libomp``.", UserWarning)


In [2]:
# solve lightgbm error on MAC
import os
os.environ['KMP_DUPLICATE_LIB_OK']='True'

In [3]:
# load data
df_train = pd.read_csv(
    RAW_DATA_DIR+'train.csv', usecols=[1, 2, 3, 4, 5],
    dtype={'onpromotion': bool},
    converters={'unit_sales': lambda u: np.log1p(
        float(u)) if float(u) > 0 else 0},
    parse_dates=["date"],
    skiprows=range(1, 66458909)  # 2016-01-01
)

df_test = pd.read_csv(
    RAW_DATA_DIR+'test.csv', usecols=[0, 1, 2, 3, 4],
    dtype={'onpromotion': bool},
    parse_dates=["date"]  # , date_parser=parser
).set_index(
    ['store_nbr', 'item_nbr', 'date']
)

items = pd.read_csv(
    RAW_DATA_DIR+'items.csv',
).set_index("item_nbr")

stores = pd.read_csv(
    RAW_DATA_DIR+'stores.csv',
).set_index("store_nbr")

### Test Period

2017-08-16 to 2017-08-31

In [4]:
test_start = date(2017, 8, 16)
test_end = date(2017,8, 31)

In [5]:
valid_start = test_start - timedelta(16)
while(1):
    if valid_start.weekday() == test_start.weekday():
        break
    valid_start = valid_start-timedelta(days=1)
valid_end = valid_start + timedelta(15)
print('valid starts from {} to {}'.format(valid_start, valid_end))

valid starts from 2017-07-26 to 2017-08-10


### Valid Period

Considering the more nearer peiods of sales data may have more in common, it would be better to find the nearest period as valid period.

Based on the analysis before, we assume the sales data is periodically with the frequency of 7 days, so we want to keep that feature same
in the train, valid and test period.

So finally, we choose valid period:

2017-07-26 to 2017-08-10


In [6]:
valid_start = date(2017, 7, 26)
valid_end = date(2017, 8, 10)

### Filter Period

#### Earthquake happended on April 16, 2016. It may affect for the next several weeks.

In [7]:
# filter the period which is affected by earthquake.
filter_date = date(2016,4,16) + timedelta(7*4)
lag_max = 140
train_start=  filter_date+timedelta(days=lag_max)

while(1):
    train_start = train_start + timedelta(1)
    if train_start.weekday() == valid_start.weekday():
        break
print('train datasets starts from {}'.format(train_start))

train datasets starts from 2016-10-05


In [8]:
train_start = date(2017, 2, 8)

### Wages in the public sector are paid every two weeks on the 15 th and on the last day of the month. Supermarket sales could be affected by this.


In [9]:
df_train = df_train[df_train['date']>=filter_date]

/Users/liuyu/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: Comparing Series of datetimes with 'datetime.date'.  Currently, the
'datetime.date' is coerced to a datetime. In the future pandas will
not coerce, and a TypeError will be raised. To retain the current
behavior, convert the 'datetime.date' to a datetime with
'pd.Timestamp'.
  """Entry point for launching an IPython kernel.


#### Promo feature

In [10]:
promo_train = df_train.set_index(
    ["store_nbr", "item_nbr", "date"])[["onpromotion"]]

# missing onpromotions filling
promo_train = promo_train.unstack(level=-1).fillna(False)
promo_train.columns = promo_train.columns.get_level_values(1)

In [11]:
# missing test onpromotions filling
promo_test = df_test[["onpromotion"]].unstack(level=-1).fillna(False)
promo_test.columns = promo_test.columns.get_level_values(1)
# filter those items/stores in promo_test but not in promo_train
promo_test = promo_test.reindex(promo_train.index).fillna(False)

In [12]:
promo_features = pd.concat([promo_train, promo_test], axis=1)
del promo_test, promo_train

## Label

In [13]:
# label
df_train = df_train.set_index(["store_nbr", "item_nbr", "date"])[["unit_sales"]].unstack(level=-1).fillna(0)

### Item

In [14]:
items = items.reindex(df_train.index.get_level_values(1))

#### Item Family

In [15]:
items['family'] = items['family'].astype('category')
item_family_features = items.family.cat.codes.values

#### Item's class

In [16]:
items['class'] = items['class'].astype('category')
item_class_features = items['class'].cat.codes.values

### Store

In [17]:
stores = stores.reindex(df_train.index.get_level_values(0))

#### Store's city

In [18]:
stores['city'] = stores['city'].astype('category')
store_city_features = stores['city'].cat.codes.values

#### Store's state

In [19]:
stores['state'] = stores['state'].astype('category')
store_state_features = stores['state'].cat.codes.values

#### Store's type

In [20]:
stores['type'] = stores['type'].astype('category')
store_type_features = stores['type'].cat.codes.values

#### Store's cluster

In [21]:
stores['cluster'] = stores['cluster'].astype('category')
store_cluster_features = stores['cluster'].cat.codes.values

In [22]:
df_train.columns = df_train.columns.get_level_values(1)

#### Filling missing date

In [23]:
date_list = df_train.columns
obj_list = pd.date_range(filter_date, test_start-timedelta(1))
diff_list = list(set(obj_list) - set(date_list)) 
for i in diff_list:
    print(i)
    df_train[i] = 0

2016-12-25 00:00:00


In [24]:
date_list = promo_features.columns
obj_list = pd.date_range(filter_date, test_end)
diff_list = list(set(obj_list) - set(date_list)) 
for i in diff_list:
    print(i)
    promo_features[i] = 0

2016-12-25 00:00:00


#### Lagging and sliding windows

In [25]:
LAG_DICT = {'unit_sales': [1,2,3,4,5,6,7,14,21,28,35,42,49,56,63],
            'onpromotion': [2, 3,4,5,6, 7, 14, 21]}

SLIDING_DICT = {'unit_sales': [3, 4, 5, 6, 7, 14, 21, 30, 60, 63]}

# initialise dirs
RAW_DATA_DIR = 'datasets/'

In [26]:
def get_timespan(df, 
                 start_time,
                 minus,
                 periods,
                 freq='D'):
    return df[pd.date_range(start_time - timedelta(days=minus), periods=periods, freq=freq)]

def gen_dataset(df, 
                promo_features,
                item_family_features,
                item_class_features,
                store_city_features,
                store_state_features,
                store_type_features,
                store_cluster_features,
                start_time,
                is_train=True):
    # init
    X = pd.DataFrame()
    
    for i in LAG_DICT['unit_sales']:
        X['lag_{}_sales'.format(i)] = get_timespan(df, start_time, i, 1).values.ravel()
    
    for i in LAG_DICT['onpromotion']:
        X['sum_{}_promo'.format(i)] = get_timespan(promo_features, start_time, i, 1).sum(axis=1).ravel()

    for i in SLIDING_DICT['unit_sales']:
        X["mean_{}_sales".format(i)] = get_timespan(df, start_time, i, i).mean(axis=1).values
        X["std_{}_sales".format(i)] = get_timespan(df, start_time, i, i).std(axis=1).values

    for i in range(7):
        X['mean_4_dow{}_2017'.format(i)] = get_timespan(df, start_time, 28-i, 4, freq='7D').mean(axis=1).values
        X['mean_20_dow{}_2017'.format(i)] = get_timespan(df, start_time, 140-i, 20, freq='7D').mean(axis=1).values
        
    # for the next to-predict 16 days 
    for i in range(16):
        X["promo_{}".format(i)] = promo_features[start_time + timedelta(days=i)].values.astype(np.uint8)

    X['item_family_features'] = item_family_features

    X['item_class_features'] = item_class_features

    X['store_city_features'] = store_city_features

    X['store_state_features'] = store_state_features

    X['store_type_features'] = store_type_features

    X['store_cluster_features'] = store_cluster_features
        
    if is_train:
        y = df[pd.date_range(start_time, periods=16)].values
        return X, y
    return X


#### Generate train, valid and test sets

In [27]:
print("Preparing dataset...")

nbr_weeks = int((valid_start - train_start).days/7)

X_l, y_l = [], []

for i in tqdm(range(nbr_weeks), desc = 'No. of week'):
    delta = timedelta(days=7 * i)
    X_tmp, y_tmp = gen_dataset(
        df_train,
        promo_features,
        item_family_features,
        item_class_features,
        store_city_features,
        store_state_features,
        store_type_features,
        store_cluster_features,
        train_start + delta
    )
    X_l.append(X_tmp)
    y_l.append(y_tmp)
#     break

No. of week:   0%|          | 0/16 [00:00<?, ?it/s]

Preparing dataset...


No. of week: 100%|██████████| 16/16 [00:10<00:00,  1.50it/s]


In [28]:
X_train = pd.concat(X_l, axis=0)
y_train = np.concatenate(y_l, axis=0)
del X_l, y_l

In [29]:

X_val, y_val = gen_dataset(df_train,
                           promo_features,
                           item_family_features,
                           item_class_features,
                           store_city_features,
                           store_state_features,
                           store_type_features,
                           store_cluster_features,
                           valid_start)
X_test = gen_dataset(df_train, 
                    promo_features,
                    item_family_features,
                    item_class_features,
                    store_city_features,
                    store_state_features,
                    store_type_features,
                    store_cluster_features,
                    test_start, is_train=False)

#### Train Model

In [30]:
print("Training and predicting models...")
params = {
    'num_leaves': 2**5 - 1,
    'objective': 'regression_l2',
    'max_depth': 8,
    'min_data_in_leaf': 50,
    'learning_rate': 0.05,
    'feature_fraction': 0.75,
    'bagging_fraction': 0.75,
    'bagging_freq': 1,
    'metric': 'l2',
    'num_threads': 4
}

MAX_ROUNDS = 200
val_pred = []
test_pred = []
cate_vars = ['item_family_features',
            'item_class_features',
            'store_city_features',
            'store_state_features',
            'store_type_features',
            'store_cluster_features']

Training and predicting models...


In [31]:
for i in tqdm(range(16)):
    dtrain = lgb.Dataset(
        X_train, label=y_train[:, i],
        categorical_feature=cate_vars,
        weight=pd.concat([items["perishable"]] * nbr_weeks) * 0.25 + 1
    )
    dval = lgb.Dataset(
        X_val, label=y_val[:, i], reference=dtrain,
        weight=items["perishable"] * 0.25 + 1,
        categorical_feature=cate_vars)

    bst = lgb.train(
        params,
        dtrain,
        num_boost_round=MAX_ROUNDS,
#         verbose_eval = False,
        valid_sets=[dtrain, dval], early_stopping_rounds=50)
    print("\n".join(("%s: %.2f" % x) for x in sorted(
        zip(X_train.columns, bst.feature_importance("gain")),
        key=lambda x: x[1], reverse=True
    )))
    val_pred.append(bst.predict(
        X_val, num_iteration=bst.best_iteration or MAX_ROUNDS))
    test_pred.append(bst.predict(
        X_test, num_iteration=bst.best_iteration or MAX_ROUNDS))
    


  0%|          | 0/16 [00:00<?, ?it/s]/Users/liuyu/anaconda3/lib/python3.7/site-packages/lightgbm/basic.py:1205: UserWarning: Using categorical_feature in Dataset.
  warnings.warn('Using categorical_feature in Dataset.')
/Users/liuyu/anaconda3/lib/python3.7/site-packages/lightgbm/basic.py:762: UserWarning: categorical_feature in param dict is overridden.
  warnings.warn('categorical_feature in param dict is overridden.')


[1]	training's l2: 1.04527	valid_1's l2: 1.00397
Training until validation scores don't improve for 50 rounds.
[2]	training's l2: 0.977625	valid_1's l2: 0.938305
[3]	training's l2: 0.915068	valid_1's l2: 0.877657
[4]	training's l2: 0.859484	valid_1's l2: 0.82379
[5]	training's l2: 0.807876	valid_1's l2: 0.773734
[6]	training's l2: 0.761179	valid_1's l2: 0.728453
[7]	training's l2: 0.718861	valid_1's l2: 0.687584
[8]	training's l2: 0.680703	valid_1's l2: 0.650498
[9]	training's l2: 0.646204	valid_1's l2: 0.617178
[10]	training's l2: 0.614835	valid_1's l2: 0.586917
[11]	training's l2: 0.586478	valid_1's l2: 0.559611
[12]	training's l2: 0.561491	valid_1's l2: 0.535655
[13]	training's l2: 0.538848	valid_1's l2: 0.513894
[14]	training's l2: 0.517553	valid_1's l2: 0.493325
[15]	training's l2: 0.498306	valid_1's l2: 0.474906
[16]	training's l2: 0.480889	valid_1's l2: 0.458186
[17]	training's l2: 0.465057	valid_1's l2: 0.443102
[18]	training's l2: 0.450613	valid_1's l2: 0.429152
[19]	training'

  6%|▋         | 1/16 [01:15<18:55, 75.69s/it]

[1]	training's l2: 0.943767	valid_1's l2: 0.92528
Training until validation scores don't improve for 50 rounds.
[2]	training's l2: 0.886554	valid_1's l2: 0.869257
[3]	training's l2: 0.834857	valid_1's l2: 0.818485
[4]	training's l2: 0.787832	valid_1's l2: 0.77227
[5]	training's l2: 0.745386	valid_1's l2: 0.730549
[6]	training's l2: 0.707661	valid_1's l2: 0.693672
[7]	training's l2: 0.672799	valid_1's l2: 0.659269
[8]	training's l2: 0.641954	valid_1's l2: 0.628939
[9]	training's l2: 0.61359	valid_1's l2: 0.601086
[10]	training's l2: 0.587667	valid_1's l2: 0.575659
[11]	training's l2: 0.564615	valid_1's l2: 0.553139
[12]	training's l2: 0.543306	valid_1's l2: 0.532076
[13]	training's l2: 0.524002	valid_1's l2: 0.513121
[14]	training's l2: 0.506913	valid_1's l2: 0.496404
[15]	training's l2: 0.490965	valid_1's l2: 0.480794
[16]	training's l2: 0.476592	valid_1's l2: 0.466728
[17]	training's l2: 0.463545	valid_1's l2: 0.453943
[18]	training's l2: 0.45172	valid_1's l2: 0.442382
[19]	training's

 12%|█▎        | 2/16 [02:23<17:05, 73.29s/it]

...

 88%|████████▊ | 14/16 [14:23<01:59, 59.98s/it]

[1]	training's l2: 1.05389	valid_1's l2: 1.00847
Training until validation scores don't improve for 50 rounds.
[2]	training's l2: 0.989743	valid_1's l2: 0.946425
[3]	training's l2: 0.93163	valid_1's l2: 0.890158
[4]	training's l2: 0.881542	valid_1's l2: 0.841019
[5]	training's l2: 0.836168	valid_1's l2: 0.796772
[6]	training's l2: 0.792457	valid_1's l2: 0.754546
[7]	training's l2: 0.753063	valid_1's l2: 0.716597
[8]	training's l2: 0.717177	valid_1's l2: 0.681957
[9]	training's l2: 0.686633	valid_1's l2: 0.652303
[10]	training's l2: 0.65701	valid_1's l2: 0.623893
[11]	training's l2: 0.631831	valid_1's l2: 0.599705
[12]	training's l2: 0.609073	valid_1's l2: 0.577735
[13]	training's l2: 0.58851	valid_1's l2: 0.55787
[14]	training's l2: 0.567943	valid_1's l2: 0.538382
[15]	training's l2: 0.549325	valid_1's l2: 0.520888
[16]	training's l2: 0.53242	valid_1's l2: 0.50505
[17]	training's l2: 0.517157	valid_1's l2: 0.490712
[18]	training's l2: 0.504471	valid_1's l2: 0.478642
[19]	training's l2:

 94%|█████████▍| 15/16 [15:26<01:00, 60.90s/it]

[1]	training's l2: 0.946809	valid_1's l2: 0.938802
Training until validation scores don't improve for 50 rounds.
[2]	training's l2: 0.89314	valid_1's l2: 0.885753
[3]	training's l2: 0.84468	valid_1's l2: 0.837507
[4]	training's l2: 0.800844	valid_1's l2: 0.793835
[5]	training's l2: 0.761128	valid_1's l2: 0.754249
[6]	training's l2: 0.725177	valid_1's l2: 0.718492
[7]	training's l2: 0.692724	valid_1's l2: 0.686148
[8]	training's l2: 0.663482	valid_1's l2: 0.657051
[9]	training's l2: 0.637764	valid_1's l2: 0.631341
[10]	training's l2: 0.613623	valid_1's l2: 0.607376
[11]	training's l2: 0.591662	valid_1's l2: 0.585676
[12]	training's l2: 0.571836	valid_1's l2: 0.565907
[13]	training's l2: 0.553897	valid_1's l2: 0.548055
[14]	training's l2: 0.537691	valid_1's l2: 0.531899
[15]	training's l2: 0.5229	valid_1's l2: 0.517251
[16]	training's l2: 0.509462	valid_1's l2: 0.503944
[17]	training's l2: 0.497286	valid_1's l2: 0.491953
[18]	training's l2: 0.486204	valid_1's l2: 0.481017
[19]	training's

100%|██████████| 16/16 [16:32<00:00, 62.28s/it]


#### Generate submission

In [32]:
mse = mean_squared_error(y_val, np.array(val_pred).transpose())
mse

0.3511710550491811

In [34]:
252/1672

0.1507177033492823

In [35]:
mse = mean_squared_error(y_val, np.array(val_pred).transpose())

mlflow.set_experiment('grocery forecasting')
with mlflow.start_run(run_name='lgbm'):
    mlflow.log_param('model', 'lgbm')
    mlflow.log_param('train starts', train_start)
    mlflow.log_params(params)
    mlflow.log_param('lagging', LAG_DICT.values())
    mlflow.log_param('slidingWindows', SLIDING_DICT.values())
    mlflow.log_param('item_info', 'Yes')
    mlflow.log_param('store_info', 'Yes')
    mlflow.log_param('private score', 0.52207)
    mlflow.log_param('private rank', '15%')
    mlflow.log_param('public score', 0.51485)

    mlflow.log_metric('mse', mse)
    
print("Validation mse:", mse)

/Users/liuyu/anaconda3/lib/python3.7/site-packages/py4j/java_collections.py:13: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
  from collections import (


Validation mse: 0.3511710550491811


In [33]:
print("Making submission...")
y_test = np.array(test_pred).transpose()
df_preds = pd.DataFrame(
    y_test, index=df_train.index,
    columns=pd.date_range("2017-08-16", periods=16)
).stack().to_frame("unit_sales")
df_preds.index.set_names(["store_nbr", "item_nbr", "date"], inplace=True)

submission = df_test[["id"]].join(df_preds, how="left").fillna(0)
submission["unit_sales"] = np.clip(np.expm1(submission["unit_sales"]), 0, 1000)
submission.to_csv('lgb.csv', float_format='%.4f', index=None)

Making submission...
